<a href="https://colab.research.google.com/github/Brnn043/EmotionClassifierAI/blob/main/emotion_classifier_bert_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Emotion Classifier with Lightweight Fine-Tuning

Classify text into one of **six core emotions** using a compact and efficient AI model.

This model is built on a **pre-trained BERT** backbone and fine-tuned with **LoRA (Low-Rank Adaptation)** — a lightweight method that updates only a small portion of the model’s parameters. This makes it **fast**, **memory-efficient**, and suitable for resource-constrained environments.


### 🧠 Model Details

- **Model type:** BERT (Bidirectional Encoder Representations from Transformers)  
- **Fine-tuning method:** LoRA (Low-Rank Adaptation)  
- **Framework:** 🤗 Hugging Face Transformers + PEFT (Parameter-Efficient Fine-Tuning)


### 💬 Supported Emotions

- 😢 **Sadness**
- 😄 **Joy**
- 💖 **Love**
- 😠 **Anger**
- 😱 **Fear**
- 😲 **Surprise**


## Setup & Install Dependencies

In [ ]:
!pip install --upgrade pip --quiet

!pip install numpy==1.26.4 --quiet

!pip install --upgrade transformers datasets peft evaluate --quiet

## Load Dataset

In [2]:
from datasets import load_dataset_builder

ds_builder = load_dataset_builder("dair-ai/emotion")

print(ds_builder.info.features)
print(ds_builder.info.splits)
print(ds_builder.info.description)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'text': Value('string'), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])}
{'train': SplitInfo(name='train', num_bytes=1743533, num_examples=16000, shard_lengths=None, dataset_name='emotion'), 'validation': SplitInfo(name='validation', num_bytes=214945, num_examples=2000, shard_lengths=None, dataset_name='emotion'), 'test': SplitInfo(name='test', num_bytes=217423, num_examples=2000, shard_lengths=None, dataset_name='emotion')}



In [3]:
from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion")

In [4]:
dataset['train']['text'][:10]

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy',
 'ive been feeling a little burdened lately wasnt sure why that was',
 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny',
 'i feel as confused about life as a teenager or as jaded as a year old man',
 'i have been with petronas for years i feel that petronas has performed well and made a huge profit',
 'i feel romantic too']

In [5]:
label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

string_map = {
    "sadness": 0,
    "joy": 1,
    "love": 2,
    "anger": 3,
    "fear": 4,
    "surprise": 5
}

def label_to_string(label):
    return label_map[label]

def string_to_label(string):
    return string_map[string]

## Preprocess Dataset

In [6]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
  inputs = tokenizer(
      examples["text"],
      padding="max_length",
      truncation=True,
      max_length = 128
  )
  inputs['labels'] = examples['label']
  return inputs

encoded_dataset = dataset.map(preprocess, batched=True)

# encoded_dataset

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
print(encoded_dataset['train']['input_ids'])

Column([[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

## Loading & Evaluating a Foundation Model

In [9]:
from transformers import AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    task_type=TaskType.SEQ_CLS,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "value"]
)

model = get_peft_model(base_model, lora_config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Performing Parameter-Efficient Fine-Tuning (PEFT)

In [10]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10,
    logging_dir="./logs",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()

/tmp/ipython-input-10-2245952336.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,1.055000,0.902518
2,0.734700,0.712230
3,0.703000,0.656943


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=3000, training_loss=0.9316837213834127, metrics={'train_runtime': 752.9299, 'train_samples_per_second': 63.751, 'train_steps_per_second': 3.984, 'total_flos': 3168487784448000.0, 'train_loss': 0.9316837213834127, 'epoch': 3.0})

option 1: save model

In [11]:
model.save_pretrained("trained_model")
tokenizer.save_pretrained("trained_model")

('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/vocab.txt',
 'trained_model/added_tokens.json',
 'trained_model/tokenizer.json')

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# import shutil
# import os

# dst = "/content/drive/MyDrive/trained_model"

# shutil.copytree("trained_model", dst, dirs_exist_ok=True)

'/content/drive/MyDrive/trained_model'

option 2: load model

In [ ]:
# import shutil
# from transformers import AutoTokenizer
# from peft import AutoPeftModelForSequenceClassification

# shutil.copytree("/content/drive/MyDrive/trained_model", "trained_model", dirs_exist_ok=True)

# model = AutoPeftModelForSequenceClassification.from_pretrained("trained_model", num_labels=6)
# tokenizer = AutoTokenizer.from_pretrained("trained_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer
from peft import AutoPeftModelForSequenceClassification

# Load PEFT adapter on top of base model
model = AutoPeftModelForSequenceClassification.from_pretrained("trained_model", num_labels=6)

# Load tokenizer (either from original or saved tokenizer)
tokenizer = AutoTokenizer.from_pretrained("trained_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Quick Test

In [15]:
from transformers import pipeline

emotion_classifier = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer)

text = input()

prediction = emotion_classifier(text)[0]

pred_label_id = int(prediction["label"].split("_")[-1])
pred_emotion = label_to_string(pred_label_id)

print(f"🎯 Predicted Emotion: {pred_emotion} (score: {prediction['score']:.2f})")

Device set to use cuda:0


nice to meet you
🎯 Predicted Emotion: joy (score: 0.84)


## Evaluation

In [16]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [17]:
import torch
from tqdm import tqdm

def evaluate_model(model, dataset, tokenizer):
    model.eval()
    preds = []
    labels = []
    device = model.device

    for example in tqdm(dataset):
        inputs = tokenizer(
            example["text"],
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).item()
        preds.append(pred)
        labels.append(example["label"])

    acc = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")

    return acc, f1

In [18]:
acc_lora, f1_lora = evaluate_model(model, dataset["test"], tokenizer)
print(f"📘 Trained LoRA Model:\n - Accuracy: {acc_lora['accuracy']:.4f} | F1: {f1_lora['f1']:.4f}")

100%|██████████| 2000/2000 [00:28<00:00, 69.53it/s]


📘 Trained LoRA Model:
 - Accuracy: 0.7855 | F1: 0.7610


In [19]:
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

acc_base, f1_base = evaluate_model(base_model, dataset["test"], tokenizer)
print(f"📕 Base Model (Untrained):\n - Accuracy: {acc_base['accuracy']:.4f} | F1: {f1_base['f1']:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 2000/2000 [04:29<00:00,  7.41it/s]


📕 Base Model (Untrained):
 - Accuracy: 0.0450 | F1: 0.0286


In [20]:
import pandas as pd
from IPython.display import display, Markdown

sample_dataset = dataset["test"].select(range(5))
sample_texts = sample_dataset["text"]
true_labels = sample_dataset["label"]

def predict_labels(model, texts):
    model.eval()
    preds = []
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(model.device)
            outputs = model(**inputs)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).item()
            preds.append(pred)
    return preds

base_preds = predict_labels(base_model, sample_texts)
lora_preds = predict_labels(model, sample_texts)

def id_to_label(idx):
    return label_map[idx]

df = pd.DataFrame({
    "📝 Text": sample_texts,
    "✅ True Label": [id_to_label(l) for l in true_labels],
    "📕 Base Model Prediction": [id_to_label(p) for p in base_preds],
    "📘 LoRA Model Prediction": [id_to_label(p) for p in lora_preds],
})

display(Markdown("### 📊 **Prediction Comparison on Sample Test Cases**"))
display(df)

display(Markdown(f"""
### 🧮 **Overall Evaluation**
| Model | Accuracy | F1 Score |
|-------|----------|----------|
| 📕 Base BERT | **{acc_base['accuracy']:.4f}** | **{f1_base['f1']:.4f}** |
| 📘 LoRA Fine-tuned | **{acc_lora['accuracy']:.4f}** | **{f1_lora['f1']:.4f}** |
"""))


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### 📊 **Prediction Comparison on Sample Test Cases**

,📝 Text,✅ True Label,📕 Base Model Prediction,📘 LoRA Model Prediction
0,im feeling rather rotten so im not very ambiti...,sadness,surprise,sadness
1,im updating my blog because i feel shitty,sadness,surprise,sadness
2,i never make her separate from me because i do...,sadness,surprise,sadness
3,i left with my bouquet of red and yellow tulip...,joy,surprise,joy
4,i was feeling a little vain when i did this one,sadness,sadness,sadness



### 🧮 **Overall Evaluation**
| Model | Accuracy | F1 Score |
|-------|----------|----------|
| 📕 Base BERT | **0.0450** | **0.0286** |
| 📘 LoRA Fine-tuned | **0.7855** | **0.7610** |
